In [13]:
from pathlib import Path
from typing import List, Optional, Tuple

import gradio as gr
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from chromadb import PersistentClient
from tqdm import tqdm

from langchain_anthropic import ChatAnthropic
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage


load_dotenv(override=True)

MODEL = "claude-haiku-4-5"
DB_NAME = "preprocessed_db"
collection_name = "docs"
embedding_model = "all-MiniLM-L6-v2"
KNOWLEDGE_BASE_PATH = Path("knowledge-base")
AVERAGE_CHUNK_SIZE = 500
RETRIEVAL_K = 10


llm = ChatAnthropic(
    model=MODEL,
    temperature=0,
    max_tokens=2000,
)

In [3]:
class Result(BaseModel):
    """Represents a retrievable chunk plus metadata."""

    page_content: str
    metadata: dict


class Chunk(BaseModel):
    """Represents one generated chunk from an input document."""

    headline: str = Field(
        description="A brief heading for this chunk, typically a few words, that is most likely to be surfaced in a query"
    )
    summary: str = Field(
        description="A few sentences summarizing the content of this chunk to answer common questions"
    )
    original_text: str = Field(
        description="The original text of this chunk from the provided document, exactly as is, not changed in any way"
    )

    def as_result(self, document: dict) -> Result:
        """Convert a generated chunk into a retrievable result object."""
        metadata = {"source": document["source"], "type": document["type"]}
        page_content = f"{self.headline}\n\n{self.summary}\n\n{self.original_text}"
        return Result(page_content=page_content, metadata=metadata)


class Chunks(BaseModel):
    """Container model used for structured chunk extraction."""

    chunks: list[Chunk]


class RankOrder(BaseModel):
    """Represents reranked chunk ids in descending relevance order."""

    order: list[int] = Field(
        description="The order of relevance of chunks, from most relevant to least relevant, by chunk id number"
    )

In [4]:
def fetch_documents() -> list[dict]:
    """Load markdown documents from the knowledge base directory."""
    documents = []

    for folder in KNOWLEDGE_BASE_PATH.iterdir():
        doc_type = folder.name
        for file in folder.rglob("*.md"):
            with open(file, "r", encoding="utf-8") as file_handle:
                documents.append(
                    {
                        "type": doc_type,
                        "source": file.as_posix(),
                        "text": file_handle.read(),
                    }
                )

    print(f"Loaded {len(documents)} documents")
    return documents


def make_prompt(document: dict) -> str:
    """Build the prompt used to split one source document into overlapping chunks."""
    how_many = (len(document["text"]) // AVERAGE_CHUNK_SIZE) + 1
    return f"""
You take a document and you split the document into overlapping chunks for a KnowledgeBase.

The document is from the shared drive of a company called Insurellm.
The document is of type: {document["type"]}
The document has been retrieved from: {document["source"]}

A chatbot will use these chunks to answer questions about the company.
You should divide up the document as you see fit, being sure that the entire document is returned in the chunks - don't leave anything out.
This document should probably be split into {how_many} chunks, but you can have more or less as appropriate.
There should be overlap between the chunks as appropriate; typically about 25% overlap or about 50 words, so you have the same text in multiple chunks for best retrieval results.

For each chunk, you should provide a headline, a summary, and the original text of the chunk.
Together your chunks should represent the entire document with overlap.

Here is the document:

{document["text"]}

Respond with the chunks.
"""


def make_messages(document: dict) -> list[dict]:
    """Create message payload for the chunking LLM call."""
    return [{"role": "user", "content": make_prompt(document)}]


def process_document(document: dict) -> list[Result]:
    """Generate structured chunks for a single document and return retrievable results."""
    structured_llm = llm.with_structured_output(Chunks)
    messages = make_messages(document)
    doc_as_chunks_obj = structured_llm.invoke(messages)
    return [chunk.as_result(document) for chunk in doc_as_chunks_obj.chunks]


def create_chunks(documents: list[dict]) -> list[Result]:
    """Process all loaded documents into chunk results."""
    chunks = []
    for document in tqdm(documents):
        chunks.extend(process_document(document))
    return chunks


def create_embeddings(chunks: list[Result]) -> None:
    """Embed chunk texts and store them in a persistent ChromaDB collection."""
    chroma = PersistentClient(path=DB_NAME)
    if collection_name in [collection.name for collection in chroma.list_collections()]:
        chroma.delete_collection(collection_name)

    texts = [chunk.page_content for chunk in chunks]
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
    vectors = embeddings.embed_documents(texts)

    collection = chroma.get_or_create_collection(collection_name)
    ids = [str(index) for index in range(len(chunks))]
    metadatas = [chunk.metadata for chunk in chunks]

    collection.add(ids=ids, embeddings=vectors, documents=texts, metadatas=metadatas)
    print(f"Vectorstore created with {collection.count()} documents")

In [5]:
def fetch_context_unranked(question: str) -> list[Result]:
    """Retrieve top-k nearest chunks from ChromaDB before reranking."""
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
    query_vector = embeddings.embed_query(question)

    chroma = PersistentClient(path=DB_NAME)
    collection = chroma.get_or_create_collection(collection_name)
    results = collection.query(query_embeddings=[query_vector], n_results=RETRIEVAL_K)

    chunks = []
    for document, metadata in zip(results["documents"][0], results["metadatas"][0]):
        chunks.append(Result(page_content=document, metadata=metadata))
    return chunks


def rerank(question: str, chunks: List[Result]) -> List[Result]:
    """Rerank retrieved chunks by relevance using Claude structured output."""
    system_prompt = """
You are a document re-ranker.
You are provided with a question and a list of relevant chunks of text from a query of a knowledge base.
The chunks are provided in the order they were retrieved; this should be approximately ordered by relevance,
but you may be able to improve on that.

You must rank order the provided chunks by relevance to the question, with the most relevant chunk first.

Reply ONLY with valid JSON in the following format:
{"order": [<chunk_id_1>, <chunk_id_2>, ...]}

Include all the chunk ids you are provided with, reranked.
"""

    user_prompt_lines = [
        f"The user has asked the following question:\n\n{question}\n",
        "Order all the chunks of text by relevance to the question, from most relevant to least relevant.",
        "Here are the chunks:\n",
    ]

    for index, chunk in enumerate(chunks, start=1):
        user_prompt_lines.append(f"# CHUNK ID: {index}\n\n{chunk.page_content}\n")

    user_prompt = "\n".join(user_prompt_lines)
    messages = [
        SystemMessage(content=system_prompt.strip()),
        HumanMessage(content=user_prompt.strip()),
    ]

    structured_llm = llm.with_structured_output(RankOrder)
    rank_order = structured_llm.invoke(messages)
    order = rank_order.order
    valid_order = [index for index in order if 1 <= index <= len(chunks)]
    if not valid_order:
        return chunks
    return [chunks[index - 1] for index in valid_order]


def fetch_context(question: str) -> list[Result]:
    """Retrieve and rerank chunks for a user question."""
    chunks = fetch_context_unranked(question)
    return rerank(question, chunks)

In [10]:
def normalize_message_content(content: object) -> str:
    """Normalize chat content values from Gradio/LangChain into plain text."""
    if isinstance(content, str):
        return content.strip()

    if isinstance(content, list):
        parts = []
        for item in content:
            if isinstance(item, str):
                text = item.strip()
                if text:
                    parts.append(text)
            elif isinstance(item, dict):
                text = str(item.get("text", "")).strip()
                if text:
                    parts.append(text)
        return "\n".join(parts).strip()

    if isinstance(content, dict):
        return str(content.get("text", "")).strip()

    return str(content).strip()


def rewrite_query(question: str, history: Optional[List[dict]] = None) -> str:
    """Rewrite a user question into a focused knowledge-base retrieval query."""
    if history is None:
        history = []

    history_str = ""
    if history:
        lines = []
        for message in history:
            role = "User" if message.get("role", "").lower() == "user" else "Assistant"
            content = normalize_message_content(message.get("content", ""))
            if content:
                lines.append(f"{role}: {content}")
        history_str = "\n".join(lines)
    else:
        history_str = "(no history)"

    system_content = f"""
You are in a conversation with a user, answering questions about the company Insurellm.
You are about to look up information in a Knowledge Base to answer the user's question.

This is the history of your conversation so far with the user:
{history_str}

And this is the user's current question:
{question}

Respond only with a single, refined question that you will use to search the Knowledge Base.
It should be a VERY short specific question most likely to surface content. Focus on the question details.
Don't mention the company name unless it's a general question about the company.
IMPORTANT: Respond ONLY with the knowledgebase query, nothing else.
"""

    messages = [
        SystemMessage(content=system_content),
        HumanMessage(content="Please rewrite now."),
    ]

    response = llm.invoke(messages)
    query = response.content.strip()

    for bad_start in ["Query:", "Search:", '"', "Refined query:", "Here is the query:"]:
        if query.startswith(bad_start):
            query = query[len(bad_start):].strip()

    return query.strip('"').strip()


SYSTEM_PROMPT = """
You are a knowledgeable, friendly assistant representing the company Insurellm.
You are chatting with a user about Insurellm.
Your answer will be evaluated for accuracy, relevance and completeness, so make sure it only answers the question and fully answers it.
If you don't know the answer, say so.
For context, here are specific extracts from the Knowledge Base that might be directly relevant to the user's question:
{context}

With this context, please answer the user's question. Be accurate, relevant and complete.
"""


def make_rag_messages(question: str, history: List[dict], chunks: List[Result]) -> List[dict]:
    """Create chat messages for final answer generation using retrieved context."""
    context = "\n\n".join(
        f"Extract from {chunk.metadata['source']}:\n{chunk.page_content}" for chunk in chunks
    )
    system_prompt = SYSTEM_PROMPT.format(context=context)
    return [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": question}]


def build_langchain_messages(message_dicts: List[dict]) -> List[BaseMessage]:
    """Convert dictionary messages into LangChain message objects."""
    messages: List[BaseMessage] = []
    for message in message_dicts:
        role = message.get("role", "").lower()
        content = message.get("content", "")
        if role == "system":
            messages.append(SystemMessage(content=content))
        elif role == "user":
            messages.append(HumanMessage(content=content))
        elif role == "assistant":
            messages.append(AIMessage(content=content))
    return messages


def answer_question(question: str, history: Optional[List[dict]] = None) -> Tuple[str, List[Result]]:
    """Run full RAG flow: rewrite, retrieve, rerank, and answer."""
    if history is None:
        history = []

    query = rewrite_query(question, history)
    chunks = fetch_context(query)
    message_dicts = make_rag_messages(question, history, chunks)
    messages = build_langchain_messages(message_dicts)

    response = llm.invoke(messages)
    answer = response.content.strip()
    return answer, chunks


def answer_question_stream(question: str, history: Optional[List[dict]] = None):
    """Run full RAG flow and stream the final answer token-by-token."""
    if history is None:
        history = []

    query = rewrite_query(question, history)
    chunks = fetch_context(query)
    message_dicts = make_rag_messages(question, history, chunks)
    messages = build_langchain_messages(message_dicts)

    partial_answer = ""
    for chunk in llm.stream(messages):
        if chunk.content:
            partial_answer += chunk.content
            yield partial_answer

In [11]:
def ensure_knowledge_base_ready() -> str:
    """Ensure the ChromaDB collection exists and has indexed documents."""
    chroma = PersistentClient(path=DB_NAME)
    collection_names = [collection.name for collection in chroma.list_collections()]

    if collection_name in collection_names:
        collection = chroma.get_or_create_collection(collection_name)
        if collection.count() > 0:
            return f"Knowledge base ready with {collection.count()} chunks."

    documents = fetch_documents()
    chunks = create_chunks(documents)
    create_embeddings(chunks)

    collection = chroma.get_or_create_collection(collection_name)
    return f"Knowledge base built with {collection.count()} chunks."


def gradio_chat_handler(message: str, history: List[dict]):
    """Handle Gradio chat requests and stream RAG answers."""
    rag_history = []
    for history_item in history:
        role = history_item.get("role", "")
        content = normalize_message_content(history_item.get("content", ""))
        if role in {"user", "assistant"} and content:
            rag_history.append({"role": role, "content": content})

    for partial_answer in answer_question_stream(message, rag_history):
        yield partial_answer


def build_gradio_app() -> gr.Blocks:
    """Create the Gradio UI for the RAG chat assistant."""
    with gr.Blocks(title="Insurellm RAG Assistant") as demo:
        gr.Markdown("# Insurellm RAG Assistant")
        gr.Markdown("Ask questions about Insurellm and get streamed RAG responses.")

        status_box = gr.Textbox(label="Knowledge Base Status", interactive=False)
        init_button = gr.Button("Initialize / Refresh Knowledge Base")

        chat_interface = gr.ChatInterface(
            fn=gradio_chat_handler,
            title="RAG Chat",
            description="Responses are generated with retrieval, reranking, and streaming output.",
        )

        init_button.click(fn=ensure_knowledge_base_ready, outputs=status_box)
        demo.load(fn=ensure_knowledge_base_ready, outputs=status_box)

    return demo


demo_app = build_gradio_app()

In [14]:
demo_app.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
* To create a public link, set `share=True` in `launch()`.
